<a href="https://colab.research.google.com/github/olgOk/QCircuit/blob/master/tutorials/Deutsch_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deutsch Algorithm

by Olga Okrut

Install frameworks, and import libraries

In [1]:
!pip install tensornetwork jax jaxlib colorama qcircuit

     |████████████████████████████████| 245kB 2.6MB/s 
  Created wheel for qcircuit: filename=qcircuit-1.0.1-cp36-none-any.whl size=5988 sha256=72f6b3c70c3a14ead5f6a15edcaa882a3eeddef52a3fcd0390a2439ef7d43b66
  Stored in directory: /root/.cache/pip/wheels/a3/dd/39/d3dece77c172e493c47e48a0b28382d116ff2daad674a7270b
Successfully built qcircuit
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
from qcircuit import QCircuit as qc

Now, after we have learned how quantum gates work and how to build a quantum circuit, we will jump to the first quantum algorithm. We begin with a very simple quantum algorithm - Deutsch algorithm, named after its inventor David Deutsch, which serves as an excellent proof of the supremacy of quantum computers and algorithms over classical.


The problem Deutsch algorithm tackles can now be stated as follows. Given a black box *Uf* implementing
some unknown binary function *f* that maps {0, 1} into {0, 1}. In other words function maps from one bit of the inforamtion to one bit. We have to clasify *f* as “constant” or “balanced” function. 



Here, constant means function always outputs the same bit, i.e. f(0) = f(1) = 1 or f(0) = f(1) = 0:
![picture](https://drive.google.com/uc?id=1CytjIW8-GO1KZfybNH21XtzTFWf9iTFX)



Balanced means function outputs different bits on different inputs, i.e. f(0) != f(1):
![picture](https://drive.google.com/uc?id=1WST_QiyQ9HSR_K98HAsmIkZB8fssPOWg)

The circuit for Deutsch’s algoritm is given below. The steps for the Deutsch algorithm:

1.   Prepare two qubits, one in state `1|0> + 0|1>` and the other in state `0|0> + 1|1>` (apply *X* gate on the second qubit).

2.  Apply the Hadamard gate (*H*) on both qubits to bring them to superposition.

3. The output after the Hadamard transformation will be send through the gate *Uf*. The values of the *Uf* matrix depends on the *f(x)* function. That means that the state vector after the gate *Uf* depends on the function, e.g. constant or balanced function.

4. The output from the *Uf* transormation is send to the gates Hadarard again. It will collapse the state vector from the superposition to one of the possible state depending on the function *f(x)*. 

5. The output from the Hadamard transformation will be a two qubit register. If all four possible function values are tested, it is revealed that the final output will be either `(0, 0), (0, 1), (1, 0), or (1, 1)` with probability of 1. The output value will depend on *f(x)*. The two qubits are entangled in the end, so only one of their values can be measured. This prevents us from known exactly which *f(x)* is being used. However, the first qubit in the pair will always be 1 if the function *f(x)* is **balanced**. If *f(x)* is **constant**, the algorithm outputs 0.

![picture](https://drive.google.com/uc?id=1rGNVTM3xl6AUQ6__k7LY0eSbot4W_tZU)



Now, let's create the quantum circuit above. We will use built-in method 
```
Uf(function)
```
which translates a classcal binary function *f(x)* into a unitary matrix *U*, and applies it to the circuit. As a parameter, it takes a function that needs to be tested for being balanced or constant. I will use a set of predefined functions to show the validity of the algorithm.

In [3]:
# define binary functions. Some of them are constant, other balanced
def f1(x):
    return x

def f2(x):
    return 1

def f3(x):
    return 0

def f4(x):
    return not x

def f5(x):
    return x ** 2

def f6(x):
    return not (x ** 3)

def f7(x):
  return (x % 3 == 2)

def f8(x):
  return not (x % 3 == 2)

# check if the function constant
def is_const(func):
  deutsch = qc.QCircuit(2)
  deutsch.X(0)
  deutsch.H(1)
  deutsch.H(0)
  deutsch.Uf(func)
  deutsch.H(0)
  deutsch.H(1)

  # get output state vector
  # decide if a function constanta or balanced
  output_state = deutsch.get_state_vector()
  # print("output state = ", output_state)
  if abs(output_state[3]) == 0.+0.j:
    return True
  else:
    return False

functions = [f1, f2, f3, f4, f5, f6, f7, f8]

for func in range(len(functions)):
  print('function f{} is {}'.format(func+1, 'constant' if is_const(functions[func]) else 'balansed'))

function f1 is balansed
function f2 is constant
function f3 is constant
function f4 is balansed
function f5 is balansed
function f6 is balansed
function f7 is constant
function f8 is constant


##Deutsch-Jozsa Algorithm 

Now, let's discuss more general algorithm. What if instead of a black box function that maps one bit to one bit you need to map *n* bit to one bit? That is what Deutsch-Jozsa algorithm is about. After David Deutsch introduced his algorithm, Richard Jozsa generalized Deutsch algorithm on the system of *n* bits.


The problem Deutsch-Jozsa algorithm tackles can be stated as follows. Given a black box *Uf* implementing
some unknown binary function *f* that maps $(0, 1)^n$ into (0, 1). We have to solve the same problem as with the Deutsch algorithm: clasify *f* as a “constant” or a “balanced” function. The only difference from the previous task is that the function *f* can take two or more agruments.


The circuit for the Deutsch-Jazsa algoritm is pretty the same. However, insted of two qubits we now need to initialize *n* qubits  with ``` |0> ``` and one qubit with the value of ``` |1> ```. You can find the picture of quantum circuit below.

The steps for the Deutsch-Jozsa algorithm:

1.   Prepare *n* qubits in state `1|0> + 0|1>`, and one qubit in state `0|0> + 1|1>` (apply *X* gate on this qubit).

2.  Apply the Hadamard gate (*H*) on all qubits to bring them to superposition.

3. The output after the Hadamard transformation will be send through the gate *Uf*. The values of the *Uf* matrix depends on the *f* function. 
It means that the state vector after the *Uf* operator will depend on the function, e.g. constant or balanced function.

4. The output from the *Uf* transormation is send to the Hadamard gates again. It will collapse the state vector to one of the possible state depending on the function *f()*. 

5. The last step of the algorithm is a measurement. 
The output from the Hadamard transformation will be the register with *n+1* qubits.  The first n qubits will always be 0 if the function *f()* is **constant**, otherwise *f(x)* is **balanced**.

![picture](https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Deutsch-Jozsa_Algorithm.svg/1280px-Deutsch-Jozsa_Algorithm.svg.png)




In [7]:
# define binary functions. Some of them are constant, other balanced
def func1(x, y):
    return x ^ y
def func2(x, y, z):
    return 1

# check if the function func1 constant
def is_constf1(func):
  deutsch_jozsa = qc.QCircuit(3)
  deutsch_jozsa.X(0)
  deutsch_jozsa.H(2)
  deutsch_jozsa.H(1)
  deutsch_jozsa.H(0)
  deutsch_jozsa.Uf(func)
  deutsch_jozsa.H(2)
  deutsch_jozsa.H(1)
  deutsch_jozsa.H(0)

  # get output state vector
  # decide if a function constant or balanced
  output_state = deutsch_jozsa.get_state_vector()
  # print("output_state for func1", output_state)
  if abs(output_state[7] == 0.+0.j):
    return True
  else:
    return False

# check if the function func2 constant
def is_constf2(func):
  deutsch_jozsa = qc.QCircuit(4)
  deutsch_jozsa.X(0)
  deutsch_jozsa.H(3)
  deutsch_jozsa.H(2)
  deutsch_jozsa.H(1)
  deutsch_jozsa.H(0)
  deutsch_jozsa.Uf(func)
  deutsch_jozsa.H(3)
  deutsch_jozsa.H(2)
  deutsch_jozsa.H(1)
  deutsch_jozsa.H(0)

  # get output state vector
  # decide if a function constant or balanced
  output_state = deutsch_jozsa.get_state_vector()
  # print("output_state for func2", output_state)
  if abs(output_state[15] == 0.+0.j):
    return True
  else:
    return False

print('function func1 is {}'.format('constant' if is_constf1(func1) else 'balansed'))
print('function func2 is {}'.format('constant' if is_constf2(func2) else 'balansed'))

output_state for func1 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  1.+0.j]
function func1 is balansed
output_state for func2 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
function func2 is constant
